In [1]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input
import pandas as pd
import numpy as np


2022-12-07 12:48:51.949146: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 12:48:52.066968: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 12:48:52.066984: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-07 12:48:52.092459: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 12:48:52.784737: W tensorflow/stream_executor/platform/de

In [53]:
data = pd.read_csv('data_full.csv')

In [54]:
data.columns

Index(['Unnamed: 0', 'img', 'category', 'section', 'floral', 'graphic',
       'striped', 'embroidered', 'pleated', 'solid', 'lattice', 'long_sleeve',
       'short_sleeve', 'sleeveless', 'maxi_length', 'mini_length', 'no_dress',
       'crew_neckline', 'v_neckline', 'square_neckline', 'no_neckline',
       'denim', 'chiffon', 'cotton', 'leather', 'faux', 'knit', 'tight',
       'loose', 'conventional', 'x_1', 'y_1', 'x_2', 'y_2', 'v1', 'x1', 'y1',
       'v2', 'x2', 'y2', 'v3', 'x3', 'y3', 'v4', 'x4', 'y4', 'v5', 'x5', 'y5',
       'v6', 'x6', 'y6', 'v7', 'x7', 'y7', 'v8', 'x8', 'y8'],
      dtype='object')

In [74]:
from keras import Sequential

In [67]:

# For now this class takes in data_full.csv and the path to the images

# In the future it will receive data directly from preprocessing

# Data will be split but here I add a split_data function

# Works with data_full passed for data

class Model:
    def __init__(self, attribute,
                data: pd.DataFrame,
                img_shape: tuple):

        # Columns for each attribute are:
        self.attributes = {
                    'design': [np.r_[0,3:10], 7],     # idx 0 are data_full columns
                    'sleeves': [np.r_[0,10:13], 3],   # idx 1 are num attributes
                    'length': [np.r_[0,13:15], 2],
                    'neckline':[np.r_[0,16:20], 4],
                    'fabric': [np.r_[0,20:26], 6],
                    'fit': [np.r_[0,26:29], 3]
                }
        self.data = data.iloc[self.attributes[attribute][0]]  # only img and design columns
        self.img_shape = img_shape
        self.num_cats = self.attributes[attribute][1]
        self.model = self.instatiate_inception()
        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def split_data(self):
        # Split data into train and test
        X_train, X_test, y_train, y_test = train_test_split(
            self.data['img'], self.data.drop(columns='img'),
            test_size=0.3, random_state=2)
        return X_train, X_test, y_train, y_test

    def instatiate_inception(self):                     # Inception V3 model
        input_layer = Input(shape=(299,299,3))          # Image size (299, 299) specific to Inception V3
        inception = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_layer)
        for layer in inception.layers:
            layer.trainable = False               # Freeze layers
        model = Sequential(inception)
        model.add(Flatten())
        model.add(Den'se(500))                           # Let's play with these last layers
        if self.num_cats == 2:
            model.add(Dense(2, activation='sigmoid'))
        else:
            model.add(Dense(self.num_cats, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=[Precision(), Recall()])
        return model

    def train(self):
        history = self.model.fit(np.array(self.X_train), self.y_train,
                batch_size=16,
                epochs=50,
                verbose=1,
                callbacks=[EarlyStopping(monitor='val_loss',
                                        patience=5,
                                        restore_best_weights=True)],
                validation_split=0.2)
        return history        # Do I need to return the model?

    def test(self):
        score = self.model.evaluate(np.array(self.X_test), self.y_test, verbose=1)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        return score

    def predict(self):
        y_pred = self.model.predict(np.array(self.X_test))
        return y_pred


In [68]:
model = Model(attribute='design',
              data = data,
              img_shape=(299, 299))

In [69]:
model.model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 flatten_13 (Flatten)        (None, 131072)            0         
                                                                 
 dense_26 (Dense)            (None, 500)               65536500  
                                                                 
 dense_27 (Dense)            (None, 7)                 3507      
                                                                 
Total params: 87,342,791
Trainable params: 65,540,007
Non-trainable params: 21,802,784
_________________________________________________________________


In [70]:
model.attributes

{'design': [array([0, 3, 4, 5, 6, 7, 8, 9]), 7],
 'sleeves': [array([ 0, 10, 11, 12]), 3],
 'length': [array([ 0, 13, 14]), 2],
 'neckline': [array([ 0, 16, 17, 18, 19]), 4],
 'fabric': [array([ 0, 20, 21, 22, 23, 24, 25]), 6],
 'fit': [array([ 0, 26, 27, 28]), 3]}

In [71]:
model.data

,Unnamed: 0,img,category,section,floral,graphic,striped,embroidered,pleated,solid,...,y5,v6,x6,y6,v7,x7,y7,v8,x8,y8
0,0,Sweet_Crochet_Blouse/img_00000070.jpg,Blouse,upper,0,0,0,1,0,0,...,273.0,0.0,212.0,267.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_00000010...,Jeans,lower,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Zippered_Single-Button_Blazer/img_00000078.jpg,Blazer,upper,0,0,0,0,0,1,...,281.0,0.0,186.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Abstract_Chevron_Draped_Dress/img_00000013.jpg,Dress,full body,1,0,0,0,0,0,...,165.0,0.0,109.0,162.0,0.0,41.0,282.0,0.0,127.0,278.0
6,6,Boho_Babe_Crochet_Top/img_00000047.jpg,Top,upper,0,0,0,0,0,1,...,234.0,0.0,132.0,232.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,Colorblock-Paneled_Tee/img_00000006.jpg,Tee,upper,0,0,0,0,0,1,...,258.0,1.0,135.0,247.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,Abstract_Floral_Print_Kimono/img_00000026.jpg,Kimono,full body,1,0,0,0,0,0,...,189.0,0.0,145.0,185.0,1.0,56.0,234.0,1.0,142.0,251.0
9,9,Stripe_Trapeze_Dress/img_00000010.jpg,Dress,full body,0,0,1,0,0,0,...,166.0,0.0,153.0,163.0,0.0,70.0,286.0,0.0,165.0,290.0


In [72]:
model.img_shape

(299, 299)

In [73]:
history = model.train()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).